# Preguntas

- Cual es el numero de tarea? The doc says proyecto 1

In [206]:
"""
text = "Hola como estas, jajajaja, que haces?"
text = "The quick brown fox jumps over the lazy dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog "

wordcloud = WordCloud(width=800, height=800, background_color='white', min_font_size=3).generate(text)

plt.figure(figsize=(8,8))
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad=0)
plt.show()
"""

'\ntext = "Hola como estas, jajajaja, que haces?"\ntext = "The quick brown fox jumps over the lazy dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog dog "\n\nwordcloud = WordCloud(width=800, height=800, background_color=\'white\', min_font_size=3).generate(text)\n\nplt.figure(figsize=(8,8))\nplt.imshow(wordcloud)\nplt.axis("off")\nplt.tight_layout(pad=0)\nplt.show()\n'

Proyecto 1 - Luis Eduardo Robles Jimenez

# Minería de Texto para Turismo

## Utils

In [207]:
import os
import numpy as np
import pandas as pd
from datetime import datetime
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from sklearn import preprocessing
from nltk.tokenize import TweetTokenizer
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer

In [208]:
class TouristSpot:
    def __init__(self, path):
        self.reviews = pd.read_csv(path)
        self.spotName = path.split('/')[-1][: -4]
        self._cleanUp()
        self.nDashes = 30

    def _cleanUp(self):
        #self.reviews.columns = [c.strip() for c in self.reviews.columns]

        #if self.reviews.loc[:, 'Edad'].dtypes == object: self.reviews.loc[:, 'Edad'] = [n.strip() for n in self.reviews.loc[:, 'Edad']]
        #self.reviews.loc[:, 'Edad'] = self.reviews.loc[:, 'Edad'].replace([np.nan, ''], -1)
        #self.reviews.loc[:, 'Edad'] = self.reviews.loc[:, 'Edad'].astype('int32')

        columns = ["Título de la opinión", "Opinión"]
        for col in columns:
            desc = []
            for o in self.reviews.loc[:, col]:
                if o[0] == '"': o = o[1:]
                if o[-1] == '"': o = o[:-1]
                desc.append(o)
            self.reviews.loc[:, col] = desc

    def wordCloud(self):
        #pip install wordcloud
        pass

    def LSA(self):
        subgroups = []
        subgroups.append(self.reviews.loc[self.reviews.loc[:, 'Género'] == 'Masculino'])
        subgroups.append(self.reviews.loc[self.reviews.loc[:, 'Género'] == 'Femenino'])
        subgroups.append(self.reviews.loc[self.reviews.loc[:, 'Nacional ó Internacional'] == 'Nacional'])
        subgroups.append(self.reviews.loc[self.reviews.loc[:, 'Nacional ó Internacional'] == 'Internacional'])
        subgroups.append(np.logical_and(self.reviews.loc[:, 'Edad'] < 10, self.reviews.loc[:, 'Edad'] < 30))
        subgroups.append(np.logical_and(self.reviews.loc[:, 'Edad'] < 60, self.reviews.loc[:, 'Edad'] < 100))

        for s in subgroups:
            vectorizer = TfidfVectorizer(min_df = 1)
            bow = vectorizer.fit_transform(s)
            terms = vectorizer.get_feature_names()
            
            bow = preprocessing.normalize(bow, norm = 'l2')
            svd = TruncatedSVD(n_components = 3)
            lsa = svd.fit_transform(bow)

            for i, comp in enumerate(lsa.components_):
                print(f"Componente {i}:")
                terms_comp = zip(terms, comp)
                sorted_terms = sorted(terms_comp, key=lambda x:x[1], reverse=True)[:10]
                for t in sorted_terms:
                    print(t[0], end=',')
                print('\n')


        return




    def temporalAnalysis(self):

        self.reviews['newDate'] = ['/'.join(date.split('/')[1:]) for date in self.reviews.loc[:, 'Fecha']]

        dates = []
        bad, neutral, good, total = [], [], [], []
        for groupName, reviewsGroup in self.reviews.groupby('newDate'):
            dates.append(groupName)
            bad.append(np.sum(reviewsGroup.loc[:, 'Escala'] < 3))
            neutral.append(np.sum(reviewsGroup.loc[:, 'Escala'] == 3))
            good.append(np.sum(reviewsGroup.loc[:, 'Escala'] > 3))

        grades = np.array(bad) + np.array(neutral) + np.array(good)
        bad =       np.array(bad)       / grades * 100
        neutral =   np.array(neutral)   / grades * 100
        good =      np.array(good)      / grades * 100

        argsort = np.argsort([datetime.strptime(date, '%m/%y' if len(date) == 5 else '%m/%Y') for date in dates])[::-1]
        dates =     np.array([dates[i] for i in argsort])
        bad =       np.array([bad[i] for i in argsort])
        neutral =   np.array([neutral[i] for i in argsort])
        good =      np.array([good[i] for i in argsort])


        _, stackedChart = plt.subplots(figsize = (15, 15))
        stackedChart.barh(dates, good, label='Good reviews', color = '#00ff00')
        stackedChart.barh(dates, neutral, left = good, label='Neutral reviews', color = '#ffff00')
        stackedChart.barh(dates, bad, left = good + neutral, label='Bad reviews', color = '#ff0000')
        stackedChart.legend()
        stackedChart.set_title(f'Reviews of {self.spotName} by month')
        plt.show()

    def describe(self):
        nRows, nCols = 2, 1
        fig = plt.figure(figsize = (15, 10))
        fig.subplots_adjust(hspace = 0.5)

        print(self.nDashes * '-', self.spotName, self.nDashes * '-')

        grades = fig.add_subplot(321)
        avg = np.round(np.average(self.reviews.loc[:, 'Escala']), decimals = 2)
        sd = np.round(np.std(self.reviews.loc[:, 'Escala']), decimals = 2)
        grades.hist(self.reviews.loc[:, 'Escala'])
        grades.set_title(f"Grades (1 worst; 5 best); Average = {avg}; SD = {sd}")

        length = fig.add_subplot(322)
        lenOpinions = [len(TweetTokenizer().tokenize(o)) for o in self.reviews.loc[:, 'Opinión']]
        avg = np.round(np.average(lenOpinions), decimals = 2)
        sd = np.round(np.std(lenOpinions), decimals = 2)
        length.hist(lenOpinions)
        length.set_title(f"Length of opinions (words); Average = {avg}; SD = {sd}")

        ages = fig.add_subplot(323)
        ages.hist(self.reviews.loc[:, 'Edad'], bins = 10)
        ages.set_title(f'Ages distribution')

        visitors = fig.add_subplot(324)
        visitors.hist(self.reviews.loc[:, 'Nacional ó Internacional'])
        visitors.set_title('Local or foreigner')

        monthNames = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
        popularity = fig.add_subplot(325)
        months = [int(d.split('/')[1]) for d in self.reviews.loc[:, 'Fecha']]
        m, c = np.unique(months, return_counts = True)
        popularity.bar(m, c)
        popularity.set_title('Popularity by month')
        popularity.set_xticks(np.arange(1, len(m) + 1), labels = monthNames)

        languages = fig.add_subplot(326)
        m, c = np.unique(self.reviews.loc[:, 'Idioma'], return_counts = True)
        languages.barh(m, c)
        languages.set_title('Language')

        plt.show()

## Actividades

### 1. Preprocesamiento y stats

In [209]:
path = "../../data/guanajuato/"
spots = [TouristSpot(os.path.join(path, d)) for d in os.listdir(path)]

In [210]:
for spot in spots: 
    break
    spot.describe()
    continue

In [211]:
for spot in spots: 
    break
    spot.temporalAnalysis()
    continue

In [212]:
for spot in spots: 
    spot.LSA()
    continue
    break

AttributeError: 'TfidfVectorizer' object has no attribute 'get_feature_names'